# PARTE 1: Extração de Dados

## Configurando Ambiente

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Exibir versão do Python
import platform
platform.python_version()

In [ ]:
try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [ ]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    # URLs
    start_urls = [
        'http://quotes.toscrape.com/page/1/'
    ]
    
    # Parse da página principal a ser crawleada
    def parse(self, response):
       
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract(),
                'author': quote.css('span small::text').extract(),
                'tags': quote.css('div.tags a.tag::text').extract()
            }

In [ ]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(QuotesSpider)
process.start()

# PARTE 2: Gerando Arquivo de Saída

## Ao executar esse trecho, reinicie o jupyter notebook

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Exibir versão do Python
import platform
platform.python_version()

try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [2]:
import json

class JsonWriterPipeline(object):

    # Função para gerar/abrir arquivo JSON
    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    # Fechar arquivo após escrita
    def close_spider(self, spider):
        self.file.close()

    # Inserir itens coletados da página WEB no arquivo JSON criado
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    # URLs
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    
    # Configuração obrigatória de pipeline para geração de arquivo de saída
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'quoteresult.json'                        
    }
    
    # Parse da página principal a ser crawleada
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract()[0],
                'author': quote.css('span small::text').extract()[0],
                'tags': quote.css('div.tags a.tag::text').extract()
            }

In [4]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(QuotesSpider)
process.start()

2021-11-15 10:23:13 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-11-15 10:23:13 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-11-15 10:23:13 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-11-15 10:23:13 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}
2021-11-15 10:23:13 [py.warnings] WARNING: C:\Users\San Diego\anaconda3\lib\site-packages\scrapy\extensions\feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)



In [6]:
import pandas as pd
# Carregando JSON criado para visualizar saída
output = pd.read_json('quoteresult.jl', lines=True)
output

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,“This life is what you make it. No matter what...,Marilyn Monroe,"[friends, heartbreak, inspirational, life, lov..."


## ---------------------------------------------------------------------------------------------------------------------------------

## Outro Exemplo: site SEFAZ

In [ ]:
try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [ ]:
import json

class JsonWriterPipeline(object):

    # Função para gerar/abrir arquivo JSON
    def open_spider(self, spider):
        self.file = open('normaresult.jl', 'w')

    # Fechar arquivo após escrita
    def close_spider(self, spider):
        self.file.close()

    # Inserir itens coletados da página WEB no arquivo JSON criado
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [ ]:
class SefazItem(scrapy.Item):
    decreto = scrapy.Field()
    publicacao_dou = scrapy.Field()
    ementa = scrapy.Field()
    norma = scrapy.Field()

In [ ]:
def get_publicacao_dou(response):
    publicacao_dou = response.css('div.tituloPublicacao ::text')[0].extract().strip()
        
    return publicacao_dou


def get_ementa(response):
    # retrieve the ementa <div>
    ementa_div = response.css('p.ementa::text').extract()

    ementa_list = []
    for i in ementa_div:
        ementa_list.append(i.strip())

    # that ementa_list is broken into some sentences; the function below generates the joint text
    ementa = ' '.join(ementa_list)
    return ementa

def get_norma_elements(response):
    # retrieve the norma <div>
    norma = response.css('div.divSegmentos')

    # declaring the list
    norma_list = []
    for i in norma:
        # retrieve only the text <p> inside the <div>
        tmp = i.css('span::text')[0].extract().strip()
        # add each element into the norma list
        norma_list.append(tmp)

    return norma_list


In [ ]:
import logging

class NormaSpider(scrapy.Spider):
    name = 'norma'
    start_urls = [
        'http://normas.receita.fazenda.gov.br/sijut2consulta/link.action?visao=anotado&idAto=95072'
    ]

    # Configuração obrigatória de pipeline para geração de arquivo de saída
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'normaresult.json'                        
    }
    
    def parse(self, response):
        # Output file into json of ONE single page
        items = SefazItem()
        items['publicacao_dou'] = get_publicacao_dou(response)
        items['ementa'] = get_ementa(response)
        items['norma'] = get_norma_elements(response)

        yield items


In [ ]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(NormaSpider)
process.start()

In [ ]:
import pandas as pd
# Carregando JSON criado para visualizar saída
output = pd.read_json('normaresult.jl', lines=True)
output

In [ ]:
 pd.options.display.max_colwidth = 5000

In [ ]:
output

## Links Uteis